In [7]:
import pandas as pd
import numpy as np
import time
import os
import sys

from IPython.display import display

In [8]:
jobs = ['Data_loading_csv','Data_loading_json', 'Data_loading_parquet', 'Count_per_column', 'Mean_per_column',
        'Sum_per_column', 'Standard_deviation', 'Summary', 'Filter', 'Avg_addition_2_columns', 
        'Sum_addition_2_columns', 'Product_2_columns', 'Add_new_column', 'Add_new_column_complex_calculation', 
        'GroupBy', 'Distinct', 'Join']

dataframe_dimensions = ['1/1', '1/10', '1/100', '1/1000', '1/10000', '1/100000', '1/1000000', '1/10000000',
                   '5/1', '5/10', '5/100', '5/1000', '5/10000', '5/100000', '5/1000000', '5/10000000',
                   '10/1', '10/10', '10/100', '10/1000', '10/10000', '10/100000', '10/1000000', '10/10000000',
                   '20/1', '20/10', '20/100', '20/1000', '20/10000', '20/100000', '20/1000000', '20/10000000',
                   '30/1', '30/10', '30/100', '30/1000', '30/10000', '30/100000', '30/1000000', '30/10000000',
                   '40/1', '40/10', '40/100', '40/1000', '40/10000', '40/100000', '40/1000000', '40/10000000',]

time_df = pd.DataFrame({'Data_loading_csv':pd.Series(dtype='float'),
                        'Data_loading_json':pd.Series(dtype='float'),
                        'Data_loading_parquet':pd.Series(dtype='float'),
                        'Count_per_column':pd.Series(dtype='float'),
                        'Mean_per_column':pd.Series(dtype='float'),
                        'Median_per_column':pd.Series(dtype='float'),
                        'Max_per_column':pd.Series(dtype='float'),
                        'Min_per_column':pd.Series(dtype='float'),
                        'Sum_per_column':pd.Series(dtype='float'),
                        'Standard_deviation_per_column':pd.Series(dtype='float'),
                        'Summary':pd.Series(dtype='float'),
                        'Filter':pd.Series(dtype='float'),
                        'Avg_addition_2_columns':pd.Series(dtype='float'),
                        'Sum_addition_2_columns':pd.Series(dtype='float'),
                        'Product_addition_2_columns':pd.Series(dtype='float'),
                        'Add_new_column':pd.Series(dtype='float'),
                        'Add_new_column_comparing_size':pd.Series(dtype='float'),
                        'GroupBy':pd.Series(dtype='float'),
                        'Distinct':pd.Series(dtype='float'),
                        'Number_distinct_values':pd.Series(dtype='float'),
                        'Join_raw':pd.Series(dtype='float'),
                        'Join':pd.Series(dtype='float'),
                        }, index=dataframe_dimensions)

In [9]:
PATH = './Dataframes/'

# 6_10000000 to big - high computation time
df_support = pd.read_csv("Support_Dataframe_6_1000.csv", index_col=False) 

In [10]:
def time_measurement(function, df, function_name, dataframe_dimension, nr_repetitions=1, **kwargs):
    measured_times = []
    for i in range(nr_repetitions):
        start_time = time.time()
        ret = function(df, **kwargs)
        measured_times.append(time.time()-start_time)
    time_df.loc[dataframe_dimension, function_name] = np.mean(measured_times)
    print(f"{function_name} - {dataframe_dimension} - Time: {time_df.loc[dataframe_dimension, function_name]}")
    return ret
    
def Data_loading_csv (df, file_name):
    return pd.read_csv(PATH + file_name, index_col=False)  
    
def Data_loading_parquet(df, file_name):
    return pd.read_parquet(PATH + file_name, engine='pyarrow') #PC, Server
    #return pd.read_parquet(PATH + file_name, engine='fastparquet') #Cluster

def Data_loading_json(df, file_name):
    #reader = pd.read_json(PATH + file_name, orient='records', lines=True, chunksize=1000000)
    #for x,i in enumerate(reader,1):
    #    if x == 1:
    #        df_json = i
    #    else:
    #        df_json = df_json.append(i)
    #return df_json
    return pd.read_json(PATH + file_name, orient='records', lines=True)
    

def Count_per_column(df):
    return df.count(axis=1)

def Mean_per_column(df):
    return df.mean(axis=1)

def Median_per_column(df):
    return df.median(axis=1)

def Max_per_column(df):
    return df.max(axis=1)

def Min_per_column(df):
    return df.min(axis=1)

def Sum_per_column(df):
    return df.sum(axis=1)

def Standard_deviation_per_column(df):
    return df.std(axis=1)

def Summary(df):
    return df.describe()

def Filter(df, column, upperbound, lowerbound):
    return df[(df[column] > lowerbound) & (df[column] < upperbound)]

def Avg_addition_2_columns(df, column_1, column_2):
    return (df[column_1] + df[column_2])/2

def Sum_addition_2_columns(df, column_1, column_2):
    return df[column_1] + df[column_2]

def Product_addition_2_columns(df, column_1, column_2):
    return df[column_1] * df[column_2]

def Add_new_column(df, df_2, column_1, column_2, column_name): 
    df_2[column_name] = df[column_1] + df[column_2]
    return df_2

def Add_new_column_comparing_size(df, df_2, column_1, column_2, column_name):
    conditions = [df[column_1] > df[column_2], df[column_1] < df[column_2]]
    choices = ["1","2"]
    df_2[column_name] = np.select(conditions, choices, default='Tie')
    return df_2

def GroupBy(df, column):
    return df.groupby(column).sum()

def Distinct(df, column):
    return df[column].unique()

def Number_distinct_values(df):
    return df.nunique()

def Join_raw(df_1, df_2):
    return df_1.merge(df_2, on='id', how='left')

def Join(df_1, df_2, column):
    df_supp_1 = df_1.groupby(column).sum()
    df_supp_2 = df_2.groupby(column).sum()
    return df_supp_1.merge(df_supp_2, on='id', how='left')


In [11]:
for i in dataframe_dimensions:
    x,y = i.split('/')
    
    df = time_measurement(Data_loading_csv, None, 'Data_loading_csv', i, 1, file_name=f"Dataframe_{x}_{y}.csv")
    time_measurement(Data_loading_parquet, None, 'Data_loading_parquet', i, 1, file_name=f"Dataframe_{x}_{y}.parquet.gz")
    time_measurement(Data_loading_json, None, 'Data_loading_json', i, 1, file_name=f"Dataframe_{x}_{y}.json.gz")

    df_2 = df

    time_measurement(Count_per_column, df, 'Count_per_column', i)
    time_measurement(Mean_per_column, df, 'Mean_per_column', i)
    time_measurement(Median_per_column, df, 'Median_per_column', i)
    time_measurement(Max_per_column, df, 'Max_per_column', i)
    time_measurement(Min_per_column, df, 'Min_per_column', i)
    time_measurement(Sum_per_column, df, 'Sum_per_column', i)
    
    time_measurement(Standard_deviation_per_column, df, 'Standard_deviation_per_column', i)
    time_measurement(Summary, df, 'Summary', i)
    time_measurement(Filter, df, 'Filter', i, column='col0', upperbound=80, lowerbound=-40)
    
    if x == '1':
        time_measurement(Avg_addition_2_columns, df, 'Avg_addition_2_columns', i, column_1='col0', column_2='col0')
        time_measurement(Sum_addition_2_columns, df, 'Sum_addition_2_columns', i, column_1='col0', column_2='col0')

        time_measurement(Product_addition_2_columns, df, 'Product_addition_2_columns', i, column_1='col0', column_2='col0')
        time_measurement(Add_new_column, df, 'Add_new_column', i, df_2=df_2, column_1='col0', column_2='col0', column_name = 'Add_col')
        time_measurement(Add_new_column_comparing_size, df, 'Add_new_column_comparing_size', i, df_2=df_2, column_1='col0', column_2='col0', column_name = 'Comp_Size')
        time_measurement(GroupBy, df, 'GroupBy', i, column='id')
        time_measurement(Distinct, df, 'Distinct', i, column='col0')
    else:   
        time_measurement(Avg_addition_2_columns, df, 'Avg_addition_2_columns', i, column_1='col2', column_2='col3')
        time_measurement(Sum_addition_2_columns, df, 'Sum_addition_2_columns', i, column_1='col2', column_2='col3')

        time_measurement(Product_addition_2_columns, df, 'Product_addition_2_columns', i, column_1='col2', column_2='col3')
        time_measurement(Add_new_column, df, 'Add_new_column', i, df_2=df_2, column_1='col2', column_2='col3', column_name = 'Add_col')
        time_measurement(Add_new_column_comparing_size, df, 'Add_new_column_comparing_size', i, df_2=df_2, column_1='col2', column_2='col3', column_name = 'Comp_Size')
        time_measurement(GroupBy, df, 'GroupBy', i, column='id')
        time_measurement(Distinct, df, 'Distinct', i, column='col2')
        
    time_measurement(Number_distinct_values, df, 'Number_distinct_values', i)
    time_measurement(Join_raw, df, 'Join_raw', i, df_2=df_support)
    time_measurement(Join, df, 'Join', i, df_2=df_support, column='id')

display(time_df)
time_df.to_csv(f"./Results/PC_Pandas_Result_json_alternative.csv")

Data_loading_csv - 1/1 - Time: 0.007532835006713867
Data_loading_parquet - 1/1 - Time: 0.0029633045196533203
Data_loading_json - 1/1 - Time: 0.0037817955017089844
Count_per_column - 1/1 - Time: 0.0008180141448974609
Mean_per_column - 1/1 - Time: 0.00023603439331054688
Median_per_column - 1/1 - Time: 0.00013113021850585938
Max_per_column - 1/1 - Time: 0.00032401084899902344
Min_per_column - 1/1 - Time: 0.00026488304138183594
Sum_per_column - 1/1 - Time: 0.0002510547637939453
Standard_deviation_per_column - 1/1 - Time: 0.0001289844512939453
Summary - 1/1 - Time: 0.005166053771972656
Filter - 1/1 - Time: 0.0006620883941650391
Avg_addition_2_columns - 1/1 - Time: 0.0002627372741699219
Sum_addition_2_columns - 1/1 - Time: 0.00013113021850585938
Product_addition_2_columns - 1/1 - Time: 0.00014090538024902344
Add_new_column - 1/1 - Time: 0.0004520416259765625
Add_new_column_comparing_size - 1/1 - Time: 0.0008101463317871094
GroupBy - 1/1 - Time: 0.001960277557373047
Distinct - 1/1 - Time: 0.0

Add_new_column_comparing_size - 1/1000000 - Time: 0.15811896324157715
GroupBy - 1/1000000 - Time: 0.0884389877319336
Distinct - 1/1000000 - Time: 0.009160995483398438
Number_distinct_values - 1/1000000 - Time: 0.07309317588806152
Join_raw - 1/1000000 - Time: 1.1987309455871582
Join - 1/1000000 - Time: 0.06263113021850586
Data_loading_csv - 1/10000000 - Time: 1.21881103515625
Data_loading_parquet - 1/10000000 - Time: 0.2063579559326172


KeyboardInterrupt: 